In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_csv('train_transformed.csv')

In [ ]:
df.head(3)

In [ ]:
df['prior_year_spend'] = df['prior_year_spend'].apply(lambda x: 0 if x == 'None' else x)
df['prior_year_spend'] = df['prior_year_spend'].apply(lambda x: float(x))

In [ ]:
# for i in range(0,len(df)):
#     if df['prior_year_spend'][i] == 0 and df['prior_year_spend_external'][i] == 0:
#         df['prior_year_spend_cat'][i] = 'No Spending'
#     elif df['prior_year_spend'][i] + df['prior_year_spend_external'][i] > 100:
        
        

In [ ]:
sns.distplot(df['prior_year_spend_external'])

In [ ]:
sns.distplot(df['prior_year_spend'][df['prior_year_spend'] < 1400])

In [ ]:
df['prior_year_spend'].describe()

In [ ]:
promo_type = df['promotion_type']
conv_ind = df['conversion_ind']
ltv = df['ltv']

df = df.drop(['promotion_type', 'conversion_ind', 'ltv'], axis=1)

In [ ]:
df_dum = pd.get_dummies(df, drop_first=True)

In [ ]:
df_dum.shape

In [ ]:
df_dum.head(3)

## Three layers of models

### Layer 1: conversion_ind, binary

In [ ]:
from sklearn.preprocessing import StandardScaler
colnames = df_dum.columns
ss = StandardScaler()
df_dum_transformed = ss.fit_transform(df_dum)
df_dum_transformed = pd.DataFrame(df_dum_transformed, columns=colnames)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X_train, X_test, y_train, y_test = train_test_split(df_dum_transformed, conv_ind, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Feature extraction
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rfe = RFE(model)
fit = rfe.fit(X_train, y_train)
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

In [ ]:
feat_names = list(fit.get_feature_names_out())
feat_names

In [ ]:
# random forest

rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

rf.fit(X_train[feat_names], y_train)

y_pred = rf.predict(X_test[feat_names])

print(classification_report(y_test, y_pred))

In [ ]:
# xgb classifier

xgb = XGBClassifier()

xgb.fit(X_train[feat_names], y_train)

y_pred_gb = xgb.predict(X_test[feat_names])

print(classification_report(y_test, y_pred_gb))

xgb is the best performing model.

### Layer 2: promotion_type, multiclass classification

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(df_dum_transformed, promo_type, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Feature extraction
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rfe = RFE(model)
fit = rfe.fit(X_train, y_train)
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

In [ ]:
feat_names = list(fit.get_feature_names_out())
feat_names

In [ ]:
# logreg

logreg = LogisticRegression()

logreg.fit(X_train[feat_names], y_train)

y_pred = logreg.predict(X_test[feat_names])

print(classification_report(y_test, y_pred))

In [ ]:
# gradient boosting

gb = GradientBoostingClassifier()

gb.fit(X_train[feat_names], y_train)

y_pred = gb.predict(X_test[feat_names])

print(classification_report(y_test, y_pred))

In [ ]:
# naive bayes

nb = GaussianNB()

nb.fit(X_train[feat_names], y_train)

y_pred_nb = nb.predict(X_test[feat_names])

print(classification_report(y_test, y_pred_nb))

Naive Bayes is the best performing model for predicting the promotion type.

### Layer 3: ltv, regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(df_dum_transformed, ltv, test_size=0.2, random_state=42)

In [ ]:
# Feature extraction
model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
rfe = RFE(model)
fit = rfe.fit(X_train, y_train)
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

In [ ]:
feat_names = list(fit.get_feature_names_out())
feat_names

In [ ]:
# random forest

rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)

rf.fit(X_train[feat_names], y_train)

y_pred = rf.predict(X_test[feat_names])

print('Adjusted R2: ', 1 - (1-rf.score(X_test[feat_names], y_test))*(len(y_test)-1)/(len(y_test)-X_test[feat_names].shape[1]-1))
print('R2: ', rf.score(X_test[feat_names], y_test))

In [ ]:
# XGB regressor

from xgboost import XGBRegressor

xgb = XGBRegressor()

xgb.fit(X_train[feat_names], y_train)

y_pred = xgb.predict(X_test[feat_names])

print('Adjusted R2: ', 1 - (1-xgb.score(X_test[feat_names], y_test))*(len(y_test)-1)/(len(y_test)-X_test[feat_names].shape[1]-1))
print('R2: ', xgb.score(X_test[feat_names], y_test))

In [ ]:
# gradient boosting
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor()

gb.fit(X_train[feat_names], y_train)

y_pred_gb = gb.predict(X_test[feat_names])

print('Adjusted R2: ', 1 - (1-gb.score(X_test[feat_names], y_test))*(len(y_test)-1)/(len(y_test)-X_test[feat_names].shape[1]-1))
print('R2: ', gb.score(X_test[feat_names], y_test))

Gradient boosting is the best performing Regressor in this case.